In [1]:
#List all models in the model directory
import os 

def list_models():
    models = os.listdir('models')
    for model in models:
        if model.endswith('.pkl'):
            print(model)

if __name__ == '__main__':
    list_models()
    

cf_glm_model.pkl
cf_lda_model.pkl
cf_nn_model.pkl
cf_nn_nn_model.pkl
cf_rf_model.pkl
cf_svm_model.pkl
cg_glm_model.pkl
cg_lda_model.pkl
cg_nn_model.pkl
cg_rf_model.pkl
cg_svm_model.pkl


In [3]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
import openmeteo_requests
import json

# =============================================================================
# 1. 歷史資料下載：利用 ERA5_land（ECMWF Seamless）資料
# =============================================================================

HIST_URL = "https://archive-api.open-meteo.com/v1/archive"

def fetch_climate_data(latitude, longitude, year, month, day, one_single_day=False):
    """
    根據指定經緯度與年份下載該年度的【歷史】日氣象資料（ERA5_land），
    回傳一個 dict，各變數皆為 numpy array（或其他對應型態）。

    注意：當請求年份為當年時，結束日期(end_date)會自動設定為昨天，
          以避免超出 API 允許的歷史資料範圍（end_date 不能超過昨天）。
    """
    # 起始日期固定為當年1月1日
    start_date = f"{year}-01-01"
    date_obj = datetime(year, month, day)
    formatted_date = date_obj.strftime("%Y-%m-%d")
    # 決定結束日期：若請求年份為當年，則結束日期設為昨天；否則用當年12月31日
    today = date.today()
    yesterday = today - timedelta(days=1)
    if year == today.year:
        end_date = yesterday.strftime("%Y-%m-%d")
    else:
        end_date = f"{year}-12-31"
    if one_single_day:
        start_date = formatted_date
        end_date = formatted_date

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": [
            "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean",
            "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean",
            "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum",
            "rain_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max",
            "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"
        ],
        "timezone": "Asia/Singapore",
        "models": "era5_seamless"
    }

    client = openmeteo_requests.Client()
    responses = client.weather_api(HIST_URL, params=params)
    response = responses[0]

    daily = response.Daily()
    elevation = response.Elevation()
    # 順序必須與 params["daily"] 中指定的順序一致
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(5).ValuesAsNumpy()
    daily_daylight_duration = daily.Variables(8).ValuesAsNumpy()
    daily_sunshine_duration = daily.Variables(9).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(10).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(11).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(12).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(13).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(14).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(15).ValuesAsNumpy()
    daily_shortwave_radiation_sum = daily.Variables(16).ValuesAsNumpy()
    daily_et0_fao_evapotranspiration = daily.Variables(17).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )}

    daily_data["elevation"] = elevation
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["daylight_duration"] = daily_daylight_duration
    daily_data["sunshine_duration"] = daily_sunshine_duration
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
    daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

    return daily_data

# =============================================================================
# 2. 預報資料下載：利用 ECMWF IFS 預報資料
# =============================================================================

def fetch_forecast_data(latitude, longitude, forecast_days=16):
    """
    利用 ECMWF IFS 模型下載預報資料，回傳一個 dict，
    日變數順序依照下列列表：
      ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean",
       "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean",
       "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum",
       "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max",
       "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"]
    """
    import requests_cache
    from retry_requests import retry
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    forecast_client = openmeteo_requests.Client(session=retry_session)
    fc_url = "https://api.open-meteo.com/v1/forecast"
    daily_vars = [
        "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean",
        "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean",
        "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum",
        "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max",
        "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"
    ]
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": daily_vars,
        "timezone": "Asia/Singapore",
        "past_days": 10,
        "forecast_days": forecast_days,
        "models": "best_match"
    }
    responses = forecast_client.weather_api(fc_url, params=params)
    response = responses[0]
    daily = response.Daily()
    forecast_data = {"date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )}
    forecast_data["temperature_2m_max"] = daily.Variables(0).ValuesAsNumpy()
    forecast_data["temperature_2m_min"] = daily.Variables(1).ValuesAsNumpy()
    forecast_data["temperature_2m_mean"] = daily.Variables(2).ValuesAsNumpy()
    forecast_data["apparent_temperature_max"] = daily.Variables(3).ValuesAsNumpy()
    forecast_data["apparent_temperature_min"] = daily.Variables(4).ValuesAsNumpy()
    forecast_data["apparent_temperature_mean"] = daily.Variables(5).ValuesAsNumpy()
    forecast_data["daylight_duration"] = daily.Variables(6).ValuesAsNumpy()
    forecast_data["sunshine_duration"] = daily.Variables(7).ValuesAsNumpy()
    forecast_data["precipitation_sum"] = daily.Variables(8).ValuesAsNumpy()
    forecast_data["rain_sum"] = daily.Variables(9).ValuesAsNumpy()
    forecast_data["precipitation_hours"] = daily.Variables(10).ValuesAsNumpy()
    forecast_data["wind_speed_10m_max"] = daily.Variables(11).ValuesAsNumpy()
    forecast_data["wind_gusts_10m_max"] = daily.Variables(12).ValuesAsNumpy()
    forecast_data["wind_direction_10m_dominant"] = daily.Variables(13).ValuesAsNumpy()
    forecast_data["shortwave_radiation_sum"] = daily.Variables(14).ValuesAsNumpy()
    forecast_data["et0_fao_evapotranspiration"] = daily.Variables(15).ValuesAsNumpy()
    return forecast_data

# =============================================================================
# 3. 針對整個日期區間進行預測
# =============================================================================
def predict_termite_over_period(latitude, longitude, forecast_days=16):
    """
    針對從當年1月1日至執行當天加上 forecast_days（例如16天）內的每一天，
    依序執行白蟻預測，並回傳每一天的各模型預測結果與 ensemble 結果。
    
    預測所用的特徵包含每日值與累積值：
      - 每日特徵（訓練時使用的）：  
        ["elevation", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean",
         "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean",
         "daylight_duration", "precipitation_sum", "rain_sum", "precipitation_hours",
         "wind_speed_10m_max", "wind_gusts_10m_max", "shortwave_radiation_sum",
         "et0_fao_evapotranspiration", "latitude", "longitude", "day"]
      - 累積特徵：累加從1月1日起的每日數值（變數依 cum_vars 定義）
    """
    global combined_df  # 方便除錯時檢查資料
    exec_date = pd.Timestamp.now().normalize()
    year = exec_date.year

    # -----------------------------
    # 下載歷史資料（ERA5_land）
    # -----------------------------
    historical_data = fetch_climate_data(latitude, longitude, year, 1, 1, one_single_day=False)
    hist_dates = pd.to_datetime(historical_data["date"]).tz_localize(None)
    hist_mask = hist_dates < exec_date
    hist_dict = {}
    for var in historical_data:
        if var == "date":
            continue
        val = historical_data[var]
        if np.isscalar(val):
            hist_dict[var] = np.full(np.sum(hist_mask), val)
        else:
            hist_dict[var] = np.array(val)[hist_mask]
    hist_df = pd.DataFrame(hist_dict)
    hist_df["date"] = hist_dates[hist_mask]
    hist_df = hist_df.dropna()  # 若有 NaN (例如更新延遲)則剔除
    hist_df.sort_values("date", inplace=True)

    # -----------------------------
    # 下載預報資料（ECMWF IFS / Best match）
    # -----------------------------
    forecast_data = fetch_forecast_data(latitude, longitude, forecast_days=forecast_days)
    fc_dates = pd.to_datetime(forecast_data["date"]).tz_localize(None)
    fc_dict = {}
    for var in forecast_data:
        if var == "date":
            continue
        fc_dict[var] = np.array(forecast_data[var])
    fc_df = pd.DataFrame(fc_dict)
    fc_df["date"] = fc_dates
    fc_df.sort_values("date", inplace=True)

    # -----------------------------
    # 串接歷史與預報資料
    # -----------------------------
    combined_df = pd.concat([hist_df, fc_df], ignore_index=True)
    combined_df.drop_duplicates(subset="date", keep="first", inplace=True)
    combined_df.sort_values("date", inplace=True)
    combined_df.reset_index(drop=True, inplace=True)
    # 填補預報中 elevation 為 NaN 的情形
    combined_df["elevation"] = combined_df["elevation"].fillna(combined_df["elevation"].iloc[0])
    
    start_date = pd.Timestamp(f"{year}-01-01")
    end_date = exec_date + pd.Timedelta(days=forecast_days)
    combined_df = combined_df[(combined_df["date"] >= start_date) & (combined_df["date"] <= end_date)]
    combined_df.reset_index(drop=True, inplace=True)

    # -----------------------------
    # 計算累積特徵（從1月1日開始累加）
    # -----------------------------
    cum_vars = ["temperature_2m_mean", "apparent_temperature_mean", "daylight_duration",
                "sunshine_duration", "precipitation_sum", "rain_sum", "precipitation_hours",
                "shortwave_radiation_sum", "et0_fao_evapotranspiration"]
    for var in cum_vars:
        combined_df[f"cumulative_{var}"] = combined_df[var].cumsum()

    # -----------------------------
    # 加入每日「日數」與經緯度常數
    # -----------------------------
    combined_df["day"] = combined_df["date"].dt.dayofyear
    combined_df["latitude"] = latitude
    combined_df["longitude"] = longitude

    # -----------------------------
    # 讀取預訓練模型
    # -----------------------------
    model_files = glob.glob(os.path.join("models", "*_model.pkl"))
    models = {}
    for mf in model_files:
        model_name = os.path.splitext(os.path.basename(mf))[0]
        try:
            with open(mf, "rb") as f:
                models[model_name] = pickle.load(f)
        except Exception as e:
            print(f"讀取模型 {mf} 時發生錯誤: {e}")

    # -----------------------------
    # 逐日預測
    # -----------------------------
    expected_daily_keys = ["elevation", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean",
                           "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean",
                           "daylight_duration", "precipitation_sum", "rain_sum", "precipitation_hours",
                           "wind_speed_10m_max", "wind_gusts_10m_max", "shortwave_radiation_sum",
                           "et0_fao_evapotranspiration", "latitude", "longitude", "day"]
    expected_cum_keys = [f"cumulative_{var}" for var in cum_vars]

    results_list = []
    for idx, row in combined_df.iterrows():
        feature_dict = {k: row[k] for k in expected_daily_keys}
        for k in expected_cum_keys:
            feature_dict[k] = round(row[k],4)
        X = pd.DataFrame([feature_dict])
        individual_preds = {}
        cf_preds = []
        cg_preds = []
        for model_name, model in models.items():
            try:
                proba = model.predict_proba(X)[:, 1][0]
                individual_preds[model_name] = round(proba, 3)
                if model_name.startswith("cf_"):
                    cf_preds.append(proba)
                elif model_name.startswith("cg_"):
                    cg_preds.append(proba)
            except Exception as e:
                print(f"模型 {model_name} 在日期 {row['date'].date()} 預測時發生錯誤: {e}")
        ensemble_cf = float(np.mean(cf_preds)) if cf_preds else None
        ensemble_cg = float(np.mean(cg_preds)) if cg_preds else None
        ensemble_overall = float(np.mean(list(individual_preds.values()))) if individual_preds else None

        results_list.append({
            "date": row["date"],
            "individual_predictions": individual_preds,
            "ensemble": {
                "cf": ensemble_cf,
                "cg": ensemble_cg,
                "overall": ensemble_overall
            },
            #"features": feature_dict
            # 只保留"latitude", "longitude"
            "features": {"latitude": feature_dict["latitude"], "longitude": feature_dict["longitude"]}
        })

    return results_list

# =============================================================================
# 4. 主程式：讀取網格點並對所有經緯度點執行預報，依預報日期存成 JSON 檔
# =============================================================================

def main():
    # 讀取網格點 CSV，假設欄位名稱為 lon, lat, elevation_m
    grid_df = pd.read_csv("./1_grid_points/taiwan_grid.csv")
    # 建立一個字典，用於依預報日期彙整所有網格點預報結果
    forecasts_by_date = {}

    # 針對每一個網格點執行預報
    for index, row in grid_df.iterrows():
        lon = row["lon"]
        lat = row["lat"]
        print(f"處理網格點：lon={lon}, lat={lat}")
        try:
            results = predict_termite_over_period(lat, lon, forecast_days=16)
        except Exception as e:
            print(f"網格點 {lon}, {lat} 執行預報失敗: {e}")
            continue
        # 將此網格點每一天的預報結果依日期彙整
        for forecast in results:
            # 將日期轉為字串格式，例如 "2025-02-09"
            date_str = forecast["date"].strftime("%Y-%m-%d")
            if date_str not in forecasts_by_date:
                forecasts_by_date[date_str] = []
            # 將此筆預報結果加入列表，預報結果內含各模型預測與 ensemble 結果
            forecasts_by_date[date_str].append(forecast)

    # 建立輸出資料夾
    os.makedirs("./2_predictions", exist_ok=True)
    # 將每個日期的預報結果存成 JSON 檔
    for date_str, forecast_list in forecasts_by_date.items():
        # 若 forecast_list 中的 "date" 欄位為 Timestamp，轉成字串
        for forecast in forecast_list:
            if isinstance(forecast.get("date"), pd.Timestamp):
                forecast["date"] = forecast["date"].strftime("%Y-%m-%d")
        out_path = os.path.join("2_predictions", f"{date_str}.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(forecast_list, f, ensure_ascii=False, indent=2)
        print(f"儲存預報結果 {date_str} 至 {out_path}")

if __name__ == "__main__":
    main()


處理網格點：lon=120.25, lat=22.0


c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


處理網格點：lon=120.5, lat=22.0


c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


處理網格點：lon=120.75, lat=22.0


...

處理網格點：lon=120.75, lat=25.25


c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


處理網格點：lon=121.0, lat=25.25


c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\allen\.conda\envs\termite_model\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


儲存預報結果 2025-01-01 至 2_predictions\2025-01-01.json
儲存預報結果 2025-01-02 至 2_predictions\2025-01-02.json
儲存預報結果 2025-01-03 至 2_predictions\2025-01-03.json
儲存預報結果 2025-01-04 至 2_predictions\2025-01-04.json
儲存預報結果 2025-01-05 至 2_predictions\2025-01-05.json
儲存預報結果 2025-01-06 至 2_predictions\2025-01-06.json
儲存預報結果 2025-01-07 至 2_predictions\2025-01-07.json
儲存預報結果 2025-01-08 至 2_predictions\2025-01-08.json
儲存預報結果 2025-01-09 至 2_predictions\2025-01-09.json
儲存預報結果 2025-01-10 至 2_predictions\2025-01-10.json
儲存預報結果 2025-01-11 至 2_predictions\2025-01-11.json
儲存預報結果 2025-01-12 至 2_predictions\2025-01-12.json
儲存預報結果 2025-01-13 至 2_predictions\2025-01-13.json
儲存預報結果 2025-01-14 至 2_predictions\2025-01-14.json
儲存預報結果 2025-01-15 至 2_predictions\2025-01-15.json
儲存預報結果 2025-01-16 至 2_predictions\2025-01-16.json
儲存預報結果 2025-01-17 至 2_predictions\2025-01-17.json
儲存預報結果 2025-01-18 至 2_predictions\2025-01-18.json
儲存預報結果 2025-01-19 至 2_predictions\2025-01-19.json
儲存預報結果 2025-01-20 至 2_predictions\2025-01-20.json
